# **Experimento de generación de titulares**

# Prerrequisitos

In [ ]:
# Se instalan los paquetes necesarios
! pip install datasets transformers rouge-score nltk sentencepiece

     |████████████████████████████████| 225kB 32.5MB/s 
     |████████████████████████████████| 2.3MB 48.2MB/s 
     |████████████████████████████████| 1.2MB 45.8MB/s 
     |████████████████████████████████| 112kB 46.9MB/s 
     |████████████████████████████████| 245kB 53.0MB/s 
     |████████████████████████████████| 3.3MB 47.1MB/s 
     |████████████████████████████████| 901kB 33.3MB/s 


In [ ]:
# Se importan los paquetes necesarios
from datasets import load_metric
import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
import numpy as np
nltk.download('punkt')
from bs4 import BeautifulSoup
import re,string,unicodedata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Se cargan las métricas ROUGE
metric = load_metric("rouge")

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

# Carga y preprocesado de los datos

In [ ]:
# Se conecta con Google Drive para la carga de datos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Se carga el fichero con los datos en un Dataframe de pandas
df = pd.read_csv('/content/drive/MyDrive/Datasets/larazon_publico/data_larazon_publico.csv')

In [ ]:
df.describe

<bound method NDFrame.describe of        Unnamed: 0  ...                                            titular
0               0  ...                         el submarino s-80 ya flota
1               1  ...  calviño y calvo alaban (sin darse cuenta) la g...
2               2  ...  el geo de la policía tendrá una nueva sede en ...
3               3  ...  la madre de las niñas "sobran las palabras par...
4               4  ...  sólo quedan 10 presos de eta por recibir el be...
...           ...  ...                                                ...
58420       26943  ...  bruselas abre un expediente a españa por no de...
58421       26944  ...  aprobado el proyecto de ley para que las mujer...
58422       26945  ...  la comisión del alvia arranca escuchando a la ...
58423       26946  ...  erc y pdecat piden explicaciones a interior po...
58424       26947  ...  el congreso aplaza la primera sesión de contro...

[58425 rows x 5 columns]>

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)


In [ ]:
df = df.dropna()

In [ ]:
df.describe

<bound method NDFrame.describe of        Unnamed: 0  ...                                            titular
39926        8449  ...  el congreso tramitará la ley de igualdad labor...
38490        7013  ...  atlantia se quedará con cellnex y venderá hisp...
18579       18579  ...  el juez suspende la declaración de rato por fr...
44673       13196  ...  podemos, nacionalistas e independentistas pide...
53968       22491  ...  pablo iglesias fracasa en su intento "in extre...
...           ...  ...                                                ...
32186         709  ...  más de 1,3 millones de familias necesitan ayud...
41607       10130  ...  la dirección de cs ignora las críticas interna...
47520       16043  ...  la fiscalía pide el archivo de la denuncia de ...
24655       24655  ...  el traspaso de prisiones al gobierno vasco inq...
40463        8986  ...  el obispo munilla propone ayunar como protesta...

[58424 rows x 5 columns]>

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 0.1,indi,cuerpo,titular
39926,8449,8449,116,el congreso ha respaldado este lunes la tramit...,el congreso tramitará la ley de igualdad labor...
38490,7013,7013,3066,atlantia no prevé vender activos de abertis en...,atlantia se quedará con cellnex y venderá hisp...
18579,18579,18579,4683,el juez antonio serrano-arnal ha suspendido la...,el juez suspende la declaración de rato por fr...
44673,13196,13196,4589,"podemos, erc, junts, la cup, pnv, eh bildu, bn...","podemos, nacionalistas e independentistas pide..."
53968,22491,22491,13962,"pese a estar de baja por paternidad, el líder ...","pablo iglesias fracasa en su intento ""in extre..."


In [ ]:
# Se selecciona el modelo pre-entrenado
model_checkpoint = "google/mt5-small"

In [ ]:
# Al emplear AutoTokenizer la librería selecciona automáticamente un tokenizer
# adecuado para el modelo seleccionado
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# as_target_tokenizer hace que se usen los tokens adecuados para identificar
# las predicciones
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[30273, 261, 714, 1371, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
# Si se emplea un modelo T5, se añade como prefijo el nombre de la tarea
if "t5" in model_checkpoint:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
# El DataFrame se convierte a formato Dataset (requerido por la implementación)
larazon_publico_dataset = Dataset.from_pandas(df)

# Se divide en dos datasets: uno de entrenamiento y uno de validación
larazon_publico_datasets = larazon_publico_dataset.train_test_split(test_size=0.1)

In [ ]:
# Se establece el límite de tamaño para el cuerpo de las noticias y para
# los titulares (input y target)

max_input_length = 1024
max_target_length = 128

# Esta función prepara los datos en el formato que espera el modelo. 
# model_inputs incluye los cuerpos tokenizados y model_inputs["labels"] los 
# titulares tokenizados.

def preprocess_function_larazon_publico(examples):
    inputs = [prefix + doc for doc in examples["cuerpo"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    
    # Se tokenizan como target las etiquetas (titulares)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["titular"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Se aplica la función anterior a los datasets
tokenized_datasets = larazon_publico_datasets.map(preprocess_function_larazon_publico, batched=True)

# Configuración del Trainer y entrenamiento

In [ ]:
# Al emplear AutoModel la librería carga la versión adecuada del modelo
# para la tarea especificada
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# Se definen los parámetros para el entrenamiento
batch_size = 6
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
#    fp16=True,
#    fp16_opt_level='O3'
)

In [ ]:
# El objeto DataCollator se emplea para formar los batches (conjuntos de 
# ejemplos que se entrenan de una vez) a partir de los datasets.
# En este caso, se realiza padding de manera dinámica (rellenar con tokens 
# hasta max_input_length )

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# Se define una función para calcular las métricas.

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Sustituye -100 en las etiquetas (no se puede decodificar)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge exige que cada oración esté en una línea diferente
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Se añade la media de los textos generados
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# Se definen los parámetros del objeto trainer. 
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import torch

In [ ]:
# Se comprueba el servicio de procesamiento. Si no hay GPU disponibles, se 
# recomienda conectar el entorno de ejecución a una. 
# (Entorno de ejecución -> Cambiar tipo de entorno de ejecución -> GPU (acelerador de hardware))

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'Hay {torch.cuda.device_count()} GPU(s) disponibles.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No hay GPU disponibles, usando la CPU.')
    device = torch.device("cpu")

Hay 1 GPU(s) disponibles.
Device name: Tesla P100-PCIE-16GB


In [ ]:
# Se limpia la caché de cuda antes de comenzar el entrenamiento #############
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False) ###

In [ ]:
# Se realiza el entrenamiento
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.915300,1.615933,43.260000,27.636900,39.800900,39.830800,17.809900
2,1.735700,1.529293,44.028800,28.291300,40.542600,40.558700,17.874200


TrainOutput(global_step=17528, training_loss=2.009677486950706, metrics={'train_runtime': 13150.3551, 'train_samples_per_second': 1.333, 'total_flos': 310472230788864.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 808689664, 'init_mem_gpu_alloc_delta': 1200707584, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -318017536, 'train_mem_gpu_alloc_delta': 3613413888, 'train_mem_cpu_peaked_delta': 373284864, 'train_mem_gpu_peaked_delta': 8260194816})

In [ ]:
# Se guarda el modelo con todos sus pesos 
trainer.save_model("./drive/MyDrive/Modelos/lrp_mt5_small_E85_b")

# Predicciones

In [ ]:
# Se carga el modelo guardado
loaded_model = AutoModelForSeq2SeqLM.from_pretrained('./drive/MyDrive/Modelos/lrp_mt5_small_E85_b')

In [ ]:
df.loc[1]['cuerpo']

'este viernes, el presidente del gobierno, pedro sánchez, y las cuatro vicepresidentas, carmen calvo, nadia calviño, yolanda díaz y teresa ribera, se han reunido con el rey, felipe vi, para presentarles el plan de recuperación, transformación y resiliencia previsto por el ejecutivo, el paquete de reformas e inversiones que pretende impulsar españa para canalizar los 140.000 millones de fondos europeos de reconstrucción que recibirá hasta 2026.según ha informado casa real, el encuentro tuvo lugar para dar los detalles del contenido del plan al monarca, en una reunión que no estaba incluida en la agenda del rey ni del jefe del ejecutivo. en dicho encuentro, las cámaras captaron una conversación entre las vicepresidentas y el rey en las que alababan que el teatro real de madrid haya estado abierto durante la pandemia.en el vídeo publicado por casa real, se puede escuchar dicha conversación entre las vicepresidentas y el monarca, que sucede en tono distendido, al parecer antes de comenzar 

In [ ]:
df.loc[1]['titular']

'calviño y calvo alaban (sin darse cuenta) la gestión de ayuso en madrid con el rey ante un sánchez atónito'

In [ ]:
# Se tokeniza la entrada
inputs = tokenizer.encode("summarize: " + df.loc[1]['cuerpo'], return_tensors="pt", max_length=1024, truncation=True)

In [ ]:
# Se genera la salida a partir del modelo cargado
outputs = loaded_model.generate(
    inputs, 
    max_length=128, 
    min_length=10, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[     0,    259,    263,   1301,  10320,    259,    276,    317,  89083,
            303,    584,   4658,    278,    450,    362,  73069,    435,  32985,
           1838,    362,   2127,    269,  40919,    931,    259,    276,    584,
         106059,   1729,      1]])
<pad> sánchez y calvo se reúnen con el rey para presentarles el plan de recuperación y resiliencia</s>


In [ ]:
df.loc[30002]['cuerpo']

'la anulación del voto de un diputado del congreso, que había pedido votar telemáticamente y lo ha hecho presencialmente, ha provocado un empate en el escrutinio del dictamen sobre reactivación económica que obligará a repetir la votación en el próximo pleno. en una resolución de la presidenta del congreso, meritxell batet, se ordena la repetición de esa votación que este miércoles había sido favorable al contenido del dictamen con medidas y recomendaciones para la recuperación económica tras la pandemia de coronavirus. al terminar el pleno, los letrados han comprobado que un miembro de la cámara -cuya identidad no ha sido facilitada- y que había solicitado votación telemática ha votado presencialmente. los servicios jurídicos han considerado que este voto no puede contabilizarse y deben corregirse los resultados.como consecuencia de esta corrección se ha producido un empate en la votación del dictamen de recuperación económica, por lo que debe repetirse en aplicación del reglamento de

In [ ]:
df.loc[30002]['titular']

'la anulación del voto de un diputado obliga a repetir la votación del bloque económico'

In [ ]:
# Se tokeniza la entrada
inputs = tokenizer.encode("summarize: " + df.loc[30002]['cuerpo'], return_tensors="pt", max_length=1024, truncation=True)

In [ ]:
# Se genera la salida a partir del modelo cargado
outputs = loaded_model.generate(
    inputs, 
    max_length=128, 
    min_length=10, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[     0,    283,  24403,   1434,    426,    259,  57347,    269,    335,
            301,  45949,    426, 117320,    268,  25809,    335,  35840,    265,
            289,    362,    259,    265, 213539,  11611,    426,  48958,   1647,
           1615,    259, 212505,   1434,    259,  48733,      1]])
<pad> la anulación del voto de un diputado del congreso provoca un empate en el escrutinio del dictamen sobre reactivación económica</s>


In [ ]:
# Probamos ahora con una nueva noticia
df_larazon_test = pd.read_csv('/content/drive/MyDrive/Datasets/larazon/data_larazon_espana_test.csv')
df_larazon_test.columns = ['cuerpo', 'titular']
cuerpo_test = df_larazon_test.loc[7]['cuerpo']
titular_test = df_larazon_test.loc[7]['titular']

In [ ]:
print(titular_test)

Rivera planta a Ciudadanos por Casado en su convención


In [ ]:
print(cuerpo_test)

La dirección de Ciudadanos (Cs) fía su suerte a la convención que han convocado en julio para relanzar el proyecto naranja, después de los malos resultados electorales que obtuvieron en Madrid y Cataluña. Fuera de la agenda oficial dejan el debate sobre la fusión con el PP, que siguen, sin embargo, instigando desde la cúpula popular. En cierta manera, la suerte de Inés Arrimadas y de su equipo depende de si son eficaces a la hora de resucitar, en los próximos meses, «su proyecto liberal y de centro». Los precedentes y el contexto político juegan en contra. 
 Arrimadas no tendrá en esta convención el apoyo del «padre» del proyecto naranja: Albert Rivera no asistirá al cónclave en el que se juega su suerte el partido que fundó. Los hechos son que en los últimos meses no ha tenido ni una palabra de apoyo para el equipo que le relevó, después de su dimisión forzada por su fracaso en las últimas elecciones generales. Mientras, en su entorno sí airean, en privado, que está molesto con la est

In [ ]:
def preprocesado(texto):
  # Se eliminan las etiquetas HTML mediante la librería BeautifulSoup
  soup = BeautifulSoup(texto, "html.parser")
  # Se convierte a minúsculas
  texto = soup.get_text().lower()
  #Se eliminan los enlaces
  texto = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', texto)
  # Unificar comillas
  texto = re.sub(r'([\'\“\”])', r'"', texto)
  # Se eliminan caracteres especiales (se sustituyen por un espacio)
  texto = re.sub(r'([\\\/\;\:\|•«\n])', ' ', texto)  
  # Se eliminan los espacios consecutivos (se han generado algunos previamente)
  texto = re.sub(r'\s+', ' ', texto).strip()
  return texto

In [ ]:
cuerpo_test_limpio = preprocesado(cuerpo_test)
print(cuerpo_test_limpio)

la dirección de ciudadanos (cs) fía su suerte a la convención que han convocado en julio para relanzar el proyecto naranja, después de los malos resultados electorales que obtuvieron en madrid y cataluña. fuera de la agenda oficial dejan el debate sobre la fusión con el pp, que siguen, sin embargo, instigando desde la cúpula popular. en cierta manera, la suerte de inés arrimadas y de su equipo depende de si son eficaces a la hora de resucitar, en los próximos meses, su proyecto liberal y de centro». los precedentes y el contexto político juegan en contra. arrimadas no tendrá en esta convención el apoyo del padre» del proyecto naranja albert rivera no asistirá al cónclave en el que se juega su suerte el partido que fundó. los hechos son que en los últimos meses no ha tenido ni una palabra de apoyo para el equipo que le relevó, después de su dimisión forzada por su fracaso en las últimas elecciones generales. mientras, en su entorno sí airean, en privado, que está molesto con la estrateg

In [ ]:
# Se tokeniza la entrada
inputs = tokenizer.encode("summarize: " + cuerpo_test_limpio, return_tensors="pt", max_length=1024, truncation=True)

In [ ]:
# Se genera la salida a partir del modelo cargado
outputs = loaded_model.generate(
    inputs, 
    max_length=128, 
    min_length=10, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[     0,    259,  84444,    337,    259,    276,   2911,    537,    303,
           1218, 107889,    289,   3482,    269,    283,    742,  75077,    450,
            362,    259,   3758,      1]])
<pad> ciudadanos y casado se juegan en contra de la fusión con el pp</s>


In [ ]:
# OTRA NOTICIA
cuerpo_test_2 = df_larazon_test.loc[10]['cuerpo']
titular_test_2 = df_larazon_test.loc[10]['titular']

In [ ]:
print(titular_test_2)

El conflicto en Oriente Medio abre otra brecha entre Podemos y Sánchez


In [ ]:
print(cuerpo_test_2)

Silencio absoluto del presidente del Gobierno sobre lo que está sucediendo en Oriente Medio. El presidente Pedro Sánchez, muy partidario de comentar cualquier aspecto de la realidad en las redes sociales, no se ha pronunciado sobre la nueva escalada de tensión entre Israel y Palestina.Sí lo ha hecho la ministra de Exteriores, Arancha González Laya, que reivindicó el sábado «la voz fuerte» de Europa en la resolución del conflicto, que además deberá ir de la mano de la «negociación y el diálogo». Hasta aquí la implicación de la parte mayoritaria del Gobierno de coalición que parece pasar de puntillas sobre lo que está sucediendo en el ámbito internacional ya que desde que tomó posesión, la influencia de España es mejorable.Mientras el PSOE no toma partido, sus aliados en el Ejecutivo son transparentes y claros en su posición respecto a Oriente Medio. Exigen una condena a Israel por defenderse del terrorismo islámico. Así, otra vez, Podemos va por libre en la estrategia diplomática del Ej

In [ ]:
cuerpo_test_limpio_2 = preprocesado(cuerpo_test_2)

In [ ]:
# Se tokeniza la entrada
inputs = tokenizer.encode("summarize: " + cuerpo_test_limpio_2, return_tensors="pt", max_length=1024, truncation=True)

In [ ]:
# Se genera la salida a partir del modelo cargado
outputs = loaded_model.generate(
    inputs, 
    max_length=128, 
    min_length=10, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[    0,   259,   263,  1301, 10320,   375,   303, 99630,  1615,   362,
         31320,   268,   289, 24769,   265,  9458,     1]])
<pad> sánchez no se pronuncia sobre el conflicto en oriente medio</s>


In [ ]:
# OTRA NOTICIA
cuerpo_test_3 = df_larazon_test.loc[12]['cuerpo']
titular_test_3 = df_larazon_test.loc[12]['titular']

In [ ]:
print(titular_test_3)

Trapero vuelve a Madrid para una cumbre policial con el Ministerio del Interior


In [ ]:
print(cuerpo_test_3)

El secretario de Estado de Seguridad Rafael Pérez presidirá el martes, en el Ministerio del Interior, una cumbre a la que asistirán los principales mandos de Guardia civil y Policía; y de los Mossos D’ Esquadra por la Generalitat de Cataluña, con la presencia del jefe de este cuerpo policial, José Luis Trapero. Vuelve a la capital de España, pero no a la Audiencia Nacional, donde fue absuelto del delito de rebelión, sino como máximo representante de seguridad. Ya estuvo en la Comunidad de Madrid para asistir al acto, en Valdemoro, de destrucción de armas que se incautaron a bandas terroristas, pero en este caso se trrata de una reunión operativa.La cumbre tendrá lugar por la mañana y el objetivo principal es el intercambio de información en materias tan sensibles como la lucha antiterrorista, la delincuencia organizada (sobre todo grupos itinerantes), narcotráfico, ciberdelincuencia, etcétera, según han informado a LA RAZÓN fuentes policiales. 
 Estarán presentes los Directores Adjunto

In [ ]:
cuerpo_test_limpio_3 = preprocesado(cuerpo_test_3)

In [ ]:
# Se tokeniza la entrada
inputs = tokenizer.encode("summarize: " + cuerpo_test_limpio_3, return_tensors="pt", max_length=1024, truncation=True)

In [ ]:
# Se genera la salida a partir del modelo cargado
outputs = loaded_model.generate(
    inputs, 
    max_length=128, 
    min_length=10, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[     0,    259, 127053,    701,    421,  35188, 102816,  84614,    573,
            317, 103394,    259,    262,    283,    319,  84687,    266,   4008,
            595,  25566,    263,    674,   1513,    269,    259, 125617,   9242,
            259,    276,  24774,   1703,      1]])
<pad> rafael pérez presidirá una cumbre a la que asistirán los principales mandos de guardia civil y policía</s>
